### Training of a simple policy using the custom reward function

In [1]:
import sys

sys.path.append("../venv/")
!source ../venv/bin/activate

In [2]:
import numpy as np 
import pandas as pd

import math
import os

In [3]:
import gym
gym.__version__

'0.21.0'

In [4]:
from citylearn.citylearn import CityLearnEnv

In [5]:
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper

In [6]:
from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import TensorBoardOutputFormat

In [7]:
sys.path.append("../custom_reward")
from custom_reward import CustomReward

#### Create the environment

A function and a wrapper class as given in the local evaluation script provided by the challenge

In [8]:
class WrapperEnv:
    """
    Env to wrap provide Citylearn Env data without providing full env
    Preventing attribute access outside of the available functions
    """
    def __init__(self, env_data):
        self.observation_names = env_data['observation_names']
        self.action_names = env_data['action_names']
        self.observation_space = env_data['observation_space']
        self.action_space = env_data['action_space']
        self.time_steps = env_data['time_steps']
        self.seconds_per_time_step = env_data['seconds_per_time_step']
        self.random_seed = env_data['random_seed']
        self.buildings_metadata = env_data['buildings_metadata']
        self.episode_tracker = env_data['episode_tracker']
    
    def get_metadata(self):
        return {'buildings': self.buildings_metadata}

def create_citylearn_env(schema_path, reward_function, central_agent):
    env = CityLearnEnv(schema=schema_path, reward_function=reward_function, central_agent=central_agent)

    env_data = dict(
        observation_names = env.observation_names,
        action_names = env.action_names,
        observation_space = env.observation_space,
        action_space = env.action_space,
        time_steps = env.time_steps,
        random_seed = None,
        episode_tracker = None,
        seconds_per_time_step = None,
        buildings_metadata = env.get_metadata()['buildings']
    )

    wrapper_env = WrapperEnv(env_data)
    return env, wrapper_env

create environment

In [9]:
schema_path = os.path.join("../data/", "schema.json")

env, wrapper_env = create_citylearn_env(schema_path, CustomReward, True)

In [10]:
# env.get_metadata()
# env.reward_function.env_metadata

Prepare for SB3

In [11]:
env = NormalizedObservationWrapper(env)
env = StableBaselines3Wrapper(env)

Create SAC model

In [12]:
model = SAC("MlpPolicy", env, tensorboard_log="./tensorboard_logs/")

# model = SAC.load("models/custom_reward_SAC6.zip")
model.set_env(env)

/home/hkolstee/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/hkolstee/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/hkolstee/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gy

Create custom callback to track reward values

In [15]:
class CustomCallback(BaseCallback):
    """
    Custom callback for plotting additional reward values in tensorboard
    """
    def __init__(self, verbose = 0):
        super().__init__(verbose)

    def _on_rollout_end(self) -> None:
        self.logger.record("Comfort", -self.training_env.get_attr("reward_function")[0].comfort[0])
        self.logger.record("Emissions", -self.training_env.get_attr("reward_function")[0].emissions[0])
        self.logger.record("Grid", -self.training_env.get_attr("reward_function")[0].grid[0])
        self.logger.record("Resilience", -self.training_env.get_attr("reward_function")[0].resilience[0])
        self.logger.record("unmet hours of thermal comfort (u)", -self.training_env.get_attr("reward_function")[0].u[0])
        self.logger.record("carbon emissions (g)", -self.training_env.get_attr("reward_function")[0].g[0])
        self.logger.record("ramping (r)", -self.training_env.get_attr("reward_function")[0].r[0])
        self.logger.record("daily peak (d)", -self.training_env.get_attr("reward_function")[0].d[0])
        self.logger.record("load factor (l)", -self.training_env.get_attr("reward_function")[0].l[0])
        self.logger.record("all-time peak (a)", -self.training_env.get_attr("reward_function")[0].a[0])
        self.logger.record("thermal resilience (m)", -self.training_env.get_attr("reward_function")[0].m[0])
        self.logger.record("normalized unserved energy (s)", -self.training_env.get_attr("reward_function")[0].s[0])

    def _on_step(self) -> bool:
        return super()._on_step()

Train

In [16]:
# model.learn(total_timesteps = env.get_metadata()["simulation_time_steps"] * 20, 
#             log_interval = 1)
model.learn(total_timesteps = env.get_metadata()["simulation_time_steps"] * 35, 
            log_interval = 1,
            callback = CustomCallback()
            )
# model.save("models/custom_reward_SAC")

[2.05004687]
[1.58969027]
[1.029673]
[0.89902759]
[0.58619039]
[0.9135723]
[1.06940923]
[0.68339996]
[0.26940412]


/home/hkolstee/.local/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.reward_function to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.reward_function` for environment variables or `env.get_wrapper_attr('reward_function')` that will search the reminding wrappers.
  logger.warn(


[0.26803768]
[0.08109089]
[0.52580834]
[0.35669933]
[0.11060177]
[0.51660877]
[0.17830396]
[0.6102622]
[0.48237513]
[0.69800384]
[0.74659333]
[0.97740692]
[0.48703163]
[0.47198718]
[0.81164929]
[0.8465899]
[1.88018137]
[2.12583655]
[1.98361333]
[2.88265508]
[1.1245962]
[1.35937408]
[0.68801345]
[0.94544529]
[0.1637302]
[0.41483527]
[0.40899434]
[0.44577726]
[0.31827455]
[0.37140055]
[0.23988445]
[0.37262143]
[0.38286261]
[0.72620993]
[0.8694958]
[0.61224919]
[0.37959851]
[0.43439194]
[0.88309298]
[1.34852349]
[0.25021526]
[0.78501996]
[0.48225957]
[1.92694204]
[1.21560451]
[1.60570372]
[0.46586387]
[1.08225354]
[0.43421677]
[0.65290548]
[0.6358206]
[0.83320288]
[0.38208421]
[0.39561496]
[0.33354526]
[0.84524651]
[0.60848733]
[0.40985672]
[1.9033505]
[0.80285374]
[1.75913029]
[3.0004028]
[3.55372397]
[1.33947506]
[2.28358512]
[2.15280176]
[2.25077462]
[3.72217905]
[1.02992599]
[1.67673036]
[1.30129576]
[2.43385945]
[1.08356624]
[1.36694616]
[0.39768371]
[1.42184755]
[0.2515286]
[0.44321

KeyboardInterrupt: 

Evaluate 

In [ ]:
observations = env.reset()

while not env.done:
    actions, _ = model.predict(observations, deterministic=True)
    observations, _, _, _ = env.step(actions)

kpis = env.evaluate()
kpis = kpis.pivot(index='cost_function', columns='name', values='value')
kpis = kpis.dropna(how='all')
display(kpis)

name,Building_1,Building_2,Building_3,District
cost_function,,,,
annual_normalized_unserved_energy_total,0.015050,0.015185,0.014282,0.014839
annual_peak_average,NaN,NaN,NaN,1.059272
carbon_emissions_total,1.629445,1.565772,1.478256,1.557824
cost_total,1.571454,1.515609,1.445369,1.510811
daily_one_minus_load_factor_average,NaN,NaN,NaN,0.739478
daily_peak_average,NaN,NaN,NaN,1.165031
discomfort_delta_average,-7.653084,-2.596345,-2.546652,-4.265360
discomfort_delta_maximum,2.608950,5.045345,3.502832,3.719042
discomfort_delta_minimum,-12.061934,-8.127138,-5.366928,-8.518667
